# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to david.guerineau@gmail.com and will expire on November 12, 2016.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\David\AppData\Local\Temp\graphlab_server_1474798619.log.0


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [60]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [61]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [62]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 0.250179     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 0.271693     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [63]:
print model_all.get("coefficients")

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
+------------------+-------+---------------+--------+
[18 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [64]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [65]:
import numpy as np
l1_penalty_list = np.logspace(1,7, num=13)

In [66]:
all_features = ['bedrooms',
            'bathrooms',
            'sqft_living', 
            'sqft_lot', 
            'floors',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

minRSS=0
minL1 = 0
i = 0
for l1_penalty in l1_penalty_list:
    model = graphlab.linear_regression.create(training, target='price',features = all_features, l2_penalty=0.,l1_penalty=l1_penalty,validation_set=None, solver='fista', verbose=False)
    error = model.predict(validation)-validation['price']
    error2= error*error
    RSS = error2.sum()
    if i == 0:
        minRSS = RSS
        minL1 = l1_penalty
    else:
        if RSS < minRSS:
            minRSS = RSS
            minL1 = l1_penalty
    i = i+1
print minRSS
print minL1

6.00712097018e+14
10.0


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [68]:
model = graphlab.linear_regression.create(training, target='price',features = all_features, l2_penalty=0.,l1_penalty=10,validation_set=None, solver='fista', verbose=False)
error = model.predict(testing)-testing['price']
error2= error*error
RSS = error2.sum()
print RSS

1.51458351212e+14


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [69]:
print model.get("coefficients").print_rows(num_rows=14)

+---------------+-------+----------------+--------+
|      name     | index |     value      | stderr |
+---------------+-------+----------------+--------+
|  (intercept)  |  None | 18053.7874928  |  None  |
|    bedrooms   |  None | 9805.90613422  |  None  |
|   bathrooms   |  None | 32673.8610038  |  None  |
|  sqft_living  |  None | 52.0574221272  |  None  |
|    sqft_lot   |  None | 0.143804563181 |  None  |
|     floors    |  None | 30135.3476933  |  None  |
|   waterfront  |  None | 633749.919565  |  None  |
|      view     |  None | 99637.1175429  |  None  |
|   condition   |  None |  6457.6281192  |  None  |
|     grade     |  None | 7811.82668661  |  None  |
|   sqft_above  |  None | 59.9369512999  |  None  |
| sqft_basement |  None | 130.554404666  |  None  |
|    yr_built   |  None | 8.93860600602  |  None  |
|  yr_renovated |  None | 55.4144003631  |  None  |
+---------------+-------+----------------+--------+
[14 rows x 4 columns]

None


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [70]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [71]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [72]:
listnnz=[]

for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price',features = all_features, l2_penalty=0.,l1_penalty=l1_penalty,validation_set=None, solver='fista', verbose=False)
    listnnz.append([l1_penalty,model['coefficients']['value'].nnz()])
    
print listnnz

[[100000000.0, 14], [127427498.57031322, 14], [162377673.91887242, 14], [206913808.11147901, 13], [263665089.87303555, 13], [335981828.62837881, 13], [428133239.8719396, 13], [545559478.11685145, 13], [695192796.17755914, 12], [885866790.41008317, 12], [1128837891.6846883, 11], [1438449888.2876658, 10], [1832980710.8324375, 9], [2335721469.0901213, 8], [2976351441.6313128, 6], [3792690190.7322536, 4], [4832930238.5717525, 3], [6158482110.6602545, 1], [7847599703.5146227, 1], [10000000000.0, 1]]


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [73]:
l1_penalty_min = 2335721469.0901213
l1_penalty_max = 2976351441.6313128

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [74]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [79]:
minRSS=0
minL1 = 0
i = 0
j = True
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price',features = all_features, l2_penalty=0.,l1_penalty=l1_penalty,validation_set=None, solver='fista', verbose=False)
    error = model.predict(validation)-validation['price']
    error2= error*error
    RSS = error2.sum()
    if i == 0:
        minRSS = RSS
        minL1 = l1_penalty
    else:
        if model['coefficients']['value'].nnz() == max_nonzeros:
            if j:
                print model['coefficients']['value'].nnz()
                minRSS = RSS
                minL1 = l1_penalty
                j = False
            else:
                if RSS < minRSS:
                    print model['coefficients']['value'].nnz()
                    minRSS = RSS
                    minL1 = l1_penalty
    i = i+1
print minRSS
print minL1

7
8.69701252663e+14
2436873570.02


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [80]:
model = graphlab.linear_regression.create(training, target='price',features = all_features, l2_penalty=0.,l1_penalty=minL1,validation_set=None, solver='fista', verbose=False)
model.get("coefficients").print_rows(num_rows=14)

+---------------+-------+---------------+--------+
|      name     | index |     value     | stderr |
+---------------+-------+---------------+--------+
|  (intercept)  |  None | 224814.469494 |  None  |
|    bedrooms   |  None |      0.0      |  None  |
|   bathrooms   |  None | 20091.9075188 |  None  |
|  sqft_living  |  None | 43.7917121028 |  None  |
|    sqft_lot   |  None |      0.0      |  None  |
|     floors    |  None |      0.0      |  None  |
|   waterfront  |  None |      0.0      |  None  |
|      view     |  None |  28969.448843 |  None  |
|   condition   |  None |      0.0      |  None  |
|     grade     |  None | 3367.38372886 |  None  |
|   sqft_above  |  None | 42.4423181309 |  None  |
| sqft_basement |  None | 19.1673627335 |  None  |
|    yr_built   |  None |      0.0      |  None  |
|  yr_renovated |  None |      0.0      |  None  |
+---------------+-------+---------------+--------+
[14 rows x 4 columns]



In [81]:
model['coefficients']['value'].nnz()

7